# Crawling data PTA

In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

def crawl_pta(url):

    result = []
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")

    #find journal item
    journal_links = soup.find_all("li",{'data-id':'id-1'})

    #looping through journal item
    for idx,journal in enumerate(journal_links):
        journal_dict = {}

        #find journal title
        title_journal = journal.find('a',{'class':'title'}).text

        #find journal info
        info_journal = journal.find_all('span')
        writer_journal = info_journal[0].text.split(':')[1]
        supervisor1_journal = info_journal[1].text.split(':')[1]
        supervisor2_journal = info_journal[2].text.split(':')[1]

        #find journal link
        url_journal = journal.find('a',{'class':'gray button'}).get('href')

        #find journal content in url
        req_journal = requests.get(url_journal)
        soup_journal = BeautifulSoup(req_journal.text, "lxml")

        #find journal abstract
        abstract_journal = soup_journal.find("p",{'align':'justify'}).text

        #wrap in dictionary
        journal_dict['Judul'] = title_journal
        journal_dict['Penulis'] = writer_journal
        journal_dict['Pembimbing 1'] = supervisor1_journal
        journal_dict['Pembimbing 2'] = supervisor2_journal
        journal_dict['Abstrak'] = abstract_journal
        result.append(journal_dict)

    return result

data = []
for i in range (1,173):
  result = crawl_pta('https://pta.trunojoyo.ac.id/c_search/byprod/10/%d'%i)
  data += result

# data = crawl_pta('https://pta.trunojoyo.ac.id/c_search/byprod/10')

df = pd.DataFrame(data)

# saving the dataframe
df.to_csv('crawl_result.csv')